In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'C:\Users\ASUS\Desktop\DataScience\Code\Pipeline\cleanedData.csv')
df.head()

,cp,thalach,oldpeak,exang,ca,thal,slope,target,age,sex
0,0,168,1.0,0,2,3,2,0,52,1
1,0,155,3.1,1,0,3,0,0,53,1
2,0,125,2.6,1,0,3,0,0,70,1
3,0,161,0.0,0,1,3,2,0,61,1
4,0,106,1.9,0,3,2,1,0,62,0


In [3]:
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.compose import ColumnTransformer

In [36]:
df.shape

(302, 10)

In [4]:
X=df.drop('target',axis = 1)
Y= df['target']
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y, test_size=0.15,random_state=50,stratify=df['target'])

In [5]:
Xtrain

,cp,thalach,oldpeak,exang,ca,thal,slope,age,sex
44,0,165,1.0,1,2,3,1,66,0
222,0,132,2.4,1,2,3,1,60,1
87,2,165,0.2,0,0,2,1,43,0
179,1,188,0.0,0,0,2,2,44,1
143,0,96,2.2,1,1,2,0,64,1
...,...,...,...,...,...,...,...,...,...
238,2,123,0.6,0,0,2,2,51,1
96,0,177,0.0,0,1,2,2,44,1
272,0,157,2.6,0,2,3,1,60,0
81,1,140,0.0,0,0,2,2,62,1


In [6]:
trf1 = ColumnTransformer(
    [
    ('scale', RobustScaler(), ['thalach', 'age']),
    ], remainder='passthrough'
) 

In [7]:
trf2 = SelectKBest(score_func=f_classif, k='all')

In [8]:
trf3 = DecisionTreeClassifier()

In [9]:
pipe = Pipeline(
    [
        ('scale',trf1),
        ('selection',trf2),
        ('modelfit',trf3)
    ]
)

In [10]:
pipe.fit(Xtrain,Ytrain)

Pipeline(steps=[('scale',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', RobustScaler(),
                                                  ['thalach', 'age'])])),
                ('selection', SelectKBest(k='all')),
                ('modelfit', DecisionTreeClassifier())])

In [11]:
y_pred = pipe.predict(Xtest)
y_pred

array([1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1], dtype=int64)

In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(Ytest,y_pred)

0.7608695652173914

In [15]:
import pickle
pickle.dump(pipe,open('model.pkl','wb'))

In [14]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,Xtrain,Ytrain,cv=5,scoring='accuracy')

array([0.75      , 0.68627451, 0.76470588, 0.74509804, 0.64705882])